In [1]:
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchsummary import summary

from trainers import trainTorch, advTrain
from evalAttacks import evalClean, evalFGSMEnsem, evalAdvDiffModel, evalCombined

# #Different Attacks
# from evalAttacks import CWAttackRandomEnsem, CWAttackEnsem

/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iis-cvl/anaconda3/envs/advAt

/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/iis-cvl/anaconda3/envs/advAttack/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/iis-cvl

In [2]:
nb_epochs = 2
batch_size = 128
learning_rate = 0.001
train_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=True, download=True,
                    transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size)

Files already downloaded and verified


In [4]:
# Loading models from modelZoo
from modelZooCifar import CIFARNet
CiNet1 = CIFARNet()
CiNet2 = CIFARNet()
CiNet3 = CIFARNet()

ckptPath = 'cifarCkpts/'

CiNet1.load_state_dict(torch.load(ckptPath + 'CIFARNet1.pth'))
CiNet2.load_state_dict(torch.load(ckptPath + 'CIFARNet2.pth'))
CiNet3.load_state_dict(torch.load(ckptPath + 'CIFARNet3.pth'))

if torch.cuda.is_available():
    CiNet1 = CiNet1.cuda()
    CiNet2 = CiNet2.cuda()
    CiNet3 = CiNet3.cuda()

In [5]:
# Evaluate all models on clean original test data
evalClean(model1=CiNet1, test_loader=test_loader, singleModel=1)
evalClean(model1=CiNet2, test_loader=test_loader, singleModel=1)
evalClean(model1=CiNet3, test_loader=test_loader, singleModel=1)

Evaluating single model results on clean data
Clean accuracy: 75.48%
Evaluating single model results on clean data
Clean accuracy: 73.84%
Evaluating single model results on clean data
Clean accuracy: 74.93%


In [6]:
# Evaluate individual models adversarial accuracy
netModels = [CiNet1, CiNet2, CiNet3]
for i, fgsmNet in enumerate(netModels):
    for j, targetNet in enumerate(netModels):
        print('fgsm: {}, target: {}:'.format(i, j))
        evalAdvDiffModel(fgsmNet, targetNet, test_loader)
print("Evaluating model 0 with Ensemble")
evalFGSMEnsem(netModels[0], [netModels[1], netModels[2]], test_loader)
print("Evaluating model 1 with Ensemble")
evalFGSMEnsem(netModels[1], [netModels[0], netModels[2]], test_loader)
print("Evaluating model 2 with Ensemble")
evalFGSMEnsem(netModels[2], [netModels[0], netModels[1]], test_loader)

fgsm: 0, target: 0:
Evaluating the adv data with another model
Adv accuracy: 10.651％
fgsm: 0, target: 1:
Evaluating the adv data with another model
Adv accuracy: 12.490％
fgsm: 0, target: 2:
Evaluating the adv data with another model
Adv accuracy: 12.816％
fgsm: 1, target: 0:
Evaluating the adv data with another model
Adv accuracy: 11.956％
fgsm: 1, target: 1:
Evaluating the adv data with another model
Adv accuracy: 9.256％
fgsm: 1, target: 2:
Evaluating the adv data with another model
Adv accuracy: 12.658％
fgsm: 2, target: 0:
Evaluating the adv data with another model
Adv accuracy: 12.836％
fgsm: 2, target: 1:
Evaluating the adv data with another model
Adv accuracy: 13.261％
fgsm: 2, target: 2:
Evaluating the adv data with another model
Adv accuracy: 11.303％
Evaluating model 0 with Ensemble
Adv accuracy: 12.253％
Evaluating model 1 with Ensemble
Adv accuracy: 11.669％
Evaluating model 2 with Ensemble
Adv accuracy: 12.905％


In [8]:
advTrain(CiNet1, [CiNet1], train_loader)
advTrain(CiNet2, [CiNet2], train_loader)
advTrain(CiNet3, [CiNet3], train_loader)

Epoch 0, Step [125], Clean Training accuracy: 86.63%
Epoch 0, Step [125], Adv Training accuracy: 14.55%
Epoch 0, Step [250], Clean Training accuracy: 90.53%
Epoch 0, Step [250], Adv Training accuracy: 24.12%
Epoch 0, Step [375], Clean Training accuracy: 89.12%
Epoch 0, Step [375], Adv Training accuracy: 29.06%
Epoch 1, Step [500], Clean Training accuracy: 93.19%
Epoch 1, Step [500], Adv Training accuracy: 34.58%
Epoch 1, Step [625], Clean Training accuracy: 92.47%
Epoch 1, Step [625], Adv Training accuracy: 38.48%
Epoch 1, Step [750], Clean Training accuracy: 90.41%
Epoch 1, Step [750], Adv Training accuracy: 42.49%
Epoch 2, Step [875], Clean Training accuracy: 93.33%
Epoch 2, Step [875], Adv Training accuracy: 47.39%
Epoch 2, Step [1000], Clean Training accuracy: 92.89%
Epoch 2, Step [1000], Adv Training accuracy: 50.63%
Epoch 2, Step [1125], Clean Training accuracy: 92.05%
Epoch 2, Step [1125], Adv Training accuracy: 53.33%
Epoch 3, Step [1250], Clean Training accuracy: 92.77%
Epoch 

Epoch 5, Step [2000], Clean Training accuracy: 93.49%
Epoch 5, Step [2000], Adv Training accuracy: 62.53%
Epoch 5, Step [2125], Clean Training accuracy: 95.64%
Epoch 5, Step [2125], Adv Training accuracy: 64.30%
Epoch 5, Step [2250], Clean Training accuracy: 94.49%
Epoch 5, Step [2250], Adv Training accuracy: 64.83%
Epoch 6, Step [2375], Clean Training accuracy: 94.08%
Epoch 6, Step [2375], Adv Training accuracy: 64.94%
Epoch 6, Step [2500], Clean Training accuracy: 96.51%
Epoch 6, Step [2500], Adv Training accuracy: 66.81%
Epoch 6, Step [2625], Clean Training accuracy: 95.14%
Epoch 6, Step [2625], Adv Training accuracy: 66.38%
Epoch 7, Step [2750], Clean Training accuracy: 93.88%
Epoch 7, Step [2750], Adv Training accuracy: 65.97%
Epoch 7, Step [2875], Clean Training accuracy: 96.51%
Epoch 7, Step [2875], Adv Training accuracy: 68.12%
Epoch 7, Step [3000], Clean Training accuracy: 95.73%
Epoch 7, Step [3000], Adv Training accuracy: 67.57%
Epoch 7, Step [3125], Clean Training accuracy:

Epoch 9, Step [3875], Clean Training accuracy: 96.66%
Epoch 9, Step [3875], Adv Training accuracy: 75.61%
Epoch 10, Step [4000], Clean Training accuracy: 96.64%
Epoch 10, Step [4000], Adv Training accuracy: 76.17%
Epoch 10, Step [4125], Clean Training accuracy: 96.83%
Epoch 10, Step [4125], Adv Training accuracy: 75.66%
Epoch 10, Step [4250], Clean Training accuracy: 96.38%
Epoch 10, Step [4250], Adv Training accuracy: 74.63%
Epoch 11, Step [4375], Clean Training accuracy: 96.86%
Epoch 11, Step [4375], Adv Training accuracy: 76.77%
Epoch 11, Step [4500], Clean Training accuracy: 97.76%
Epoch 11, Step [4500], Adv Training accuracy: 77.06%
Epoch 11, Step [4625], Clean Training accuracy: 97.13%
Epoch 11, Step [4625], Adv Training accuracy: 77.03%
Epoch 12, Step [4750], Clean Training accuracy: 96.61%
Epoch 12, Step [4750], Adv Training accuracy: 76.94%
Epoch 12, Step [4875], Clean Training accuracy: 97.79%
Epoch 12, Step [4875], Adv Training accuracy: 77.28%
Epoch 12, Step [5000], Clean T

In [9]:
# Reevaluation after training
evalClean(model1=CiNet1, test_loader=test_loader, singleModel=1)
evalClean(model1=CiNet2, test_loader=test_loader, singleModel=1)
evalClean(model1=CiNet3, test_loader=test_loader, singleModel=1)
netModels = [CiNet1, CiNet2, CiNet3]
for i, fgsmNet in enumerate(netModels):
    for j, targetNet in enumerate(netModels):
        print('fgsm: {}, target: {}:'.format(i, j))
        evalAdvDiffModel(fgsmNet, targetNet, test_loader)
print("Evaluating model 0 with Ensemble")
evalFGSMEnsem(netModels[0], [netModels[1], netModels[2]], test_loader)
print("Evaluating model 1 with Ensemble")
evalFGSMEnsem(netModels[1], [netModels[0], netModels[2]], test_loader)
print("Evaluating model 2 with Ensemble")
evalFGSMEnsem(netModels[2], [netModels[0], netModels[1]], test_loader)

Evaluating single model results on clean data
Clean accuracy: 71.47%
Evaluating single model results on clean data
Clean accuracy: 72.09%
Evaluating single model results on clean data
Clean accuracy: 71.44%
fgsm: 0, target: 0:
Evaluating the adv data with another model
Adv accuracy: 61.620％
fgsm: 0, target: 1:
Evaluating the adv data with another model
Adv accuracy: 52.077％
fgsm: 0, target: 2:
Evaluating the adv data with another model
Adv accuracy: 53.540％
fgsm: 1, target: 0:
Evaluating the adv data with another model
Adv accuracy: 55.093％
fgsm: 1, target: 1:
Evaluating the adv data with another model
Adv accuracy: 62.104％
fgsm: 1, target: 2:
Evaluating the adv data with another model
Adv accuracy: 53.600％
fgsm: 2, target: 0:
Evaluating the adv data with another model
Adv accuracy: 54.282％
fgsm: 2, target: 1:
Evaluating the adv data with another model
Adv accuracy: 50.574％
fgsm: 2, target: 2:
Evaluating the adv data with another model
Adv accuracy: 62.184％
Evaluating model 0 with Ense

In [ ]:
# from modelZooCifar import CIFARNet
# pth = 'cifarCkpts/CIFARNet1.pth'
# target_model = CIFARNet()
# target_model = target_model.cuda()
# target_model.train()
# # opt_model = torch.optim.Adam(target_model.parameters(), lr=0.001)
# opt_model = torch.optim.SGD(target_model.parameters(), lr=0.01, momentum=0.9)
# epochs = 50
# for epoch in range(epochs):
#     loss_epoch = 0
#     if epoch == 25:
# #         opt_model = torch.optim.Adam(target_model.parameters(), lr=0.0001)
#         opt_model = torch.optim.SGD(target_model.parameters(), lr=0.005, momentum=0.9)
#     for i, data in enumerate(train_loader, 0):
#         train_imgs, train_labels = data
#         train_imgs, train_labels = train_imgs.cuda(), train_labels.cuda()
#         logits_model = target_model(train_imgs)
#         loss_model = F.cross_entropy(logits_model, train_labels)
#         loss_epoch += loss_model
#         opt_model.zero_grad()
#         loss_model.backward()
#         opt_model.step()
    
#     print('loss in epoch %d: %f' % (epoch, loss_epoch.item()))
#     if (epoch+1) % 5 == 0:
#         num_correct = 0
#         total = 0
#         for i, data in enumerate(test_loader, 0):
#             test_img, test_label = data
#             test_img, test_label = test_img.cuda(), test_label.cuda()
#             pred_lab = torch.argmax(target_model(test_img), 1)
#             num_correct += torch.sum(pred_lab==test_label,0)
#             total += len(pred_lab)
#         print('accuracy in testing set: %f\n'%(num_correct.item()/total))



# # save model
# targeted_model_file_name = pth
# torch.save(target_model.state_dict(), targeted_model_file_name)

